In [1]:
import pandas as pd
import numpy as np
import os

#### Merging 12 months of sales data into a single file

Below is how you read a single csv file into the notebook using Pandas:

In [2]:
df = pd.read_csv('./Sales_Data/Sales_April_2019.csv')

df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"


But what if you wanted to save time and read all csv files into the notebook? Do you have to have 12 lines of pd.read_csv?

There is almost always an easier, more succinct way to perform a task. Don't be afraid to Google something to find a shorter, better way.

In [5]:
files =  [file for file in os.listdir('./Sales_Data')]

for file in files:
    print(file)

.Sales_December_2019.csv.icloud
Sales_April_2019.csv
Sales_August_2019.csv
Sales_February_2019.csv
Sales_January_2019.csv
Sales_July_2019.csv
Sales_June_2019.csv
Sales_March_2019.csv
Sales_May_2019.csv
Sales_November_2019.csv
Sales_October_2019.csv
Sales_September_2019.csv


Now that we have all our files, we need to determine how to merge (or concatenate) them into a single .csv file

In [6]:
# define an empty df to store our data:
all_months_data = pd.DataFrame()

files =  [file for file in os.listdir('./Sales_Data')]
for file in files:
    df = pd.read_csv('./Sales_Data/' + file)
    all_months_data = pd.concat([all_months_data, df])

all_months_data.shape

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd3 in position 8: invalid continuation byte

In [ ]:
all_months_data.to_csv('all_data.csv', index = False)

#### Read in updated dataFrame

In [ ]:
all_data = pd.read_csv('all_data.csv')
all_data.head()

##### **QUESTION 1: What was the best month for sales? How much was earned in that month?**

**Answer:** The best month for sales is December. This is likely due to the occurrence of Christmas during the month of December -- the largest shopping holiday in the United States, annually. 

In [ ]:
all_data.columns

To find the total money earned for each month, we must calculate the total income received for each sale, then add it up for each month.

We can do this as follows:
- earned = price each * quantity of item [Done]
- sum of earned for each month
- max(sum_earned) = best month

Problems encountered:
- There exist some values that are NaN
- There exist columns that are just the column names reiterated for the sake of readability. This causes some calculations to be unable to be performed, resulting in errors.
- The digits shown in the columns are actually strings, not floats or ints

In [ ]:
df = pd.DataFrame([all_data['Quantity Ordered'], all_data['Price Each'], 
                   all_data['Order Date']])
df = np.transpose(df)

In [ ]:
df.head()

Next, we must drop the rows that only contain the column heading repeated for readability:

In [ ]:
# previously did it as follows:
''' for i in range(len(df['Quantity Ordered'])):
       if df['Quantity Ordered'][i] == 'Quantity Ordered':
          df.drop(i, inplace=True) '''

# the following does the same, but is much faster and shorter
df = df[df['Quantity Ordered'].str[:2] != 'Qu']

The above removes all entries in the DataFrame that are just the names of the column headers. This is to allow for the necessary multiplication for each month to be had.

In [ ]:
df.head()

Since the rows that show up as NaN are just blank rows across all columns for the data set, we can drop them entirely:

In [ ]:
df.dropna(inplace=True)

Now that 'Quantity Ordered' contains just digits, we must change them from Str to Floats so that we can perform arithmetic:

In [ ]:
df['Quantity Ordered'] = pd.to_numeric(df['Quantity Ordered'])

In [ ]:
df.head()

In [ ]:
df['Price Each'] = pd.to_numeric(df['Price Each'])

We must now do the same for the 'Price Each' Series. However, since we used .drop(inplace=True) previously, we don't have to repeat the step where we delete the column headings for this Series.

In [ ]:
df['Earned'] = df['Quantity Ordered'] * df['Price Each']

In [ ]:
df.head()

In order to more easily navigate the dataFrame, it is important to reset the indexes of the dataFrame so that it doesn't skip values (0, 1, 2... instead of 0, 2, 3...)

In [ ]:
df = df.reset_index(drop=True)

Now that we have the total amount earned for each sale, we must determine how much was earned per month. To do this, we must first identify the month that each purchase occured in, then we must sum all values in the 'Earned' column during that time.

Using a comprehension within a series, we're able to cut just the month portion of the order date off to add it to the new Month column.

In [ ]:
# df['Month'] = pd.Series([df['Order Date'][i][:2] for i in range(len(df['Order Date']))])

df['Month'] = pd.to_numeric(df['Order Date'].str[:2])

In [ ]:
df['Month'].unique() #ensure it's only the months and that all months are represented

In [ ]:
# [sum(df['Earned'][i]) for i in df['Month'] if i == '04']

sales_per_month = df.groupby('Month').sum()

Now that we have our total sales per month (in USD earned), we can now graph the outcome to better view the data and answer the question

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
months = np.arange(1, 13)

plt.figure(figsize = (10, 6))

plt.plot(months, sales_per_month['Earned'])
plt.xticks(months)

plt.xlabel('Months')
plt.ylabel('Sales Per Month in Millions ($USD)')
plt.title('Monthly Sales Nation-wide (2019)')

plt.show()

##### **QUESTION 2: What city sold the most products?**

**Answer:** The city that sold the most products (and earned the most in sales) is San Fransisco. This could be due to the overall population, the high frequency of technical work that may require more expensive products such as laptops, the average income within Silicon Valley that allows for more spending, etc. Similarly, the graph shows that the cities that sold the most products earned the most income, and vice versa. This shows that there aren't cities that are buying any more expensive products, but fewer products total, than any other city -- within this dataset.

In order to find the city that sold the most, we must split the data into the cities (and states), then sum up the sales that occurred in that city, similarly to how we did for the months.

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
temp = pd.Series(all_data['Purchase Address'][all_data['Purchase Address'].str[:1] != 'P'])
temp.dropna(inplace=True)
df['City'] = temp

In [ ]:
## This caused so many issues. Be sure to include 'drop = True', or else it won't work. It kept forcing NaN values back in.

df['City'] = temp.reset_index(drop=True)

In [ ]:
df.head()

###### Using the **.apply( )** method to identify cities and states

In [ ]:
def get_city_state(address):
    city = address.split(',')[1]
    state = address.split(',')[2].split(' ')[1]
    return f'{city} ({state})'

In [ ]:
x = '669 Spruce St, Los Angeles, CA 90001'

get_city_state(x)

In [ ]:
df['City'] = df['City'].apply(lambda x: get_city_state(x))

In [ ]:
df.head()

In [ ]:
results = df.groupby('City').sum()

From this, we can see that San Fransisco bought the most products, as well as earned the most in sales.

To plot this, we need to have a list of just the unique values of the cities available. However, just using the .unique() method would cause the cities to change order form the list above. In order to remedy this, we can use the following:

In [ ]:
cities = [city for city, result in df.groupby('City')] # a list comprehension for the elements in the .groupby()

In [ ]:
plt.subplots(figsize = (12, 6))

plt.bar(cities, results['Quantity Ordered'], label='Number of Products Ordered')
plt.xticks(cities, rotation='vertical', size=8)
plt.xlabel('Cities')
plt.ylabel('Products Ordered')
plt.title('Products Ordered Per City')

plt.plot(cities, results['Earned']/100, ':og', label='Earned in $10,000')
plt.legend()
plt.grid()
plt.show()

##### **QUESTION 3: What time should we display advertisements to maximize the likelihood of customers buying products?**

**Answer:** The best time to show ads is during either 12:00 pm or 11:00 pm, as these are the times that the most orders are placed. This will increase the chances of ads being seen, as this means this is the time that the most active users are interacting with the website.

In order to figure out the best time to display adds, we need to determine what time of day most orders are processed. This involves determining the orders by order ID and the time of day (hour, minute)

In [ ]:
df_orders = pd.DataFrame({
    'Order ID': all_data['Order ID'],
    'When Occured': all_data['Order Date'],
})

In [ ]:
df_orders = df_orders[df_orders['Order ID'].str[:1] != 'O']

In [ ]:
df_orders.dropna(inplace=True)
df_orders = df_orders.reset_index(drop=True)

In [ ]:
df_orders['When Occured'] = pd.to_datetime(df_orders['When Occured'])

In [ ]:
df_orders['Hours'] = df_orders['When Occured'].dt.hour
df_orders['Min'] = df_orders['When Occured'].dt.minute

In [ ]:
df_orders.head()

In [ ]:
hours = np.arange(0, 24)

In [ ]:
orders = df_orders.groupby(['Hours']).count()['Order ID']
orders

In [ ]:
plt.figure(figsize=(8, 4))

plt.plot(hours, orders)
plt.xticks(hours)

plt.title('Purchases Throughout The Day')
plt.grid()
plt.show()

##### **QUESTION 4: What products are most often sold together?**

**Answer:** The top 3 products that are most often purchased together are:
1. An iPhone & a Lightning Charging Cable: 1005 occurances
2. A Google Phone and a USB-C Charging Cable: 987 occurances
3. An iPhone and a set of Wired Headphones: 447 occurances

In order to find out which products most often sold together, we need to look for entries that share the same order ID, meaning that they were purchased at the same time. This will involve the product item name and order ID.

In [ ]:
df_together = pd.DataFrame({
    'Order': all_data['Order ID'],
    'Product': all_data['Product']
})

df_together.shape

The below is used each time to drop:
1. The rows that are just the column names
2. Blank rows (NaN entry rows)
3. Reset the index of the dataFrame so that it is sequential

In [ ]:
df_together = df_together[df_together['Order'].str[:1] != 'O']
df_together.dropna(inplace=True)
df_together = df_together.reset_index(drop=True)

For this data set, we know it is sufficiently cleaned when the x axis of the shape is 185,950

In [ ]:
df_together.shape

In [ ]:
df_together.head(20)

Now that we have the order # and the product, we need to look for duplicate order numbers to identify which products where most often purchased together. Once identified, we can put these products on the same row so as the order number so that we know they were purchased together;

In [ ]:
# together = product in product if order is duplicated()

df_together['Order'].duplicated(keep=False) #keep = False marks all as duplicates, not just the first or last ones
df_together['Grouped'] = df_together.groupby(['Order'])['Product'].transform(lambda x: ', '.join(x))

###### ***NOTE:*** If you're ever looking for a quick way to find more info about something, such as "keep = False" or ".transform( )", you can use Shift + Tab to bring up a document window that will explain more.

In [ ]:
df_together = df_together[df_together['Order'].duplicated(keep=False)] #only keep rows with more than one purchase

In [ ]:
df_together.head()

Now that we have all orders grouped up, we need only one of each order to get an accurate count:

In [ ]:
df_together = df_together[['Order', 'Grouped']].drop_duplicates()

In [ ]:
df_together.head(10)

The last step is to count the occurances of the same products being grouped:

By using itertools and collections, we can analyze this data deeper, including how many items were purchased at a time, the most and least common purchases, etc.

This solution described in reference: https://stackoverflow.com/questions/52195887/counting-unique-pairs-of-numbers-into-a-python-dictionary 

In [ ]:
from itertools import combinations
from collections import Counter

In [ ]:
count = Counter()

for row in df_together['Grouped']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list, 2))) # count the items in pairs of 2

In [ ]:
count.most_common(6) # .most_common() is a method that comes with Counter from collections

##### **QUESTION 5: What products sold the most? Why do you think?**

**Answer:** The highest selling products are items such as AAA or AA batteries, charging cables, etc. These items are cheap and relatively disposable. Batteries need to be replaced. It's not uncommon for individuals to have multiple charging cables or require a replacement cable should one get lost or damaged. These items are also significantly cheapter than other, more permenant products such as phones, computers, etc. Since this question is particularly concerned with how many of each product is purchased, it makes sense that the products that are cheap and often have to be replaced sell the most frequently. 

In order to figure out which products sold the most, we need the following: 
- Product * Quantity Ordered = total products sold

In [ ]:
df_total = pd.DataFrame({
    'Product': all_data['Product'],
    'Number Sold': all_data['Quantity Ordered']
})

#clean the data
df_total = df_total[df_total['Product'].str[:1] != 'P']
df_total.dropna(inplace=True)
df_total = df_total.reset_index(drop=True)

In [ ]:
df_total.shape

In [ ]:
df_total['Number Sold'] = pd.to_numeric(df_total['Number Sold'])

In [ ]:
df_total.head()

In [ ]:
products_sold = df_total.groupby(['Product']).sum()
products_sold

In [ ]:
products = [product for product, result in df_total.groupby('Product')]

In [ ]:
plt.figure(figsize=(12,6))

plt.plot(products, products_sold)
plt.ylabel('Number Sold')
plt.xlabel('Products')
plt.xticks(products, rotation='vertical', size=8)
plt.title('The Highest Selling Products')

plt.grid()
plt.show()

##### **BONUS QUESTION 6: What individual product makes us the most money?**

**Answer:** As hypothesized with Question 5, the products that sell most frequently bring in a very small percentage of the overall money earned through sales. The product that makes the most, The MacBook Pro, is one of the lowest selling items in terms of units sold. Similarly, the AAA battery -- the item that sold the most -- is essentially the product that brings in the least amount of monetary value. 

Interestingly enough, the product that sold the least in Question 5 (the LG Washer and Dryer) also accounts for some of the lowest monetary sales. Perhaps this is due to better alternatives, pricing, or perhaps the appeal of a washer and dryer opposed to a MacBook Pro or Phone. It may also have something to do with the lifespan of a product such as a washer or dryer, requiring very infrequently replacement or upgrade in its entirety. This is an interesting result that could use further exploration.

In order to figure out which product makes us the most money, we need to determine how many of each product is sold (already done) and multiply that by its average price to determine how much in sales each product makes up.

In [ ]:
df_sales = pd.DataFrame({
    'Item': df_total['Product'],
    'Sales': df['Earned']
})

In [ ]:
sales = df_sales.groupby(['Item']).sum()

In [ ]:
sales

In [ ]:
# using products from above;
plt.figure(figsize=(12,6))

plt.plot(products, sales)
plt.xlabel('Products')
plt.ylabel('Sales, in Millions ($USD)')
plt.xticks(products, rotation='vertical', size=8)
plt.title('The Products That Make The Most')

plt.grid()
plt.show()

rowID, OrderId, date, customerID, customerName, segment(corporate, consumer, etc), city, state, country, product, category, sales, quantity, profit